https://www.youtube.com/watch?v=8rcVcH4OTdo&list=WL&index=75&t=0s

In [1]:
import os
import os.path   # To check if the cover image file exists

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from scipy.misc import imread, imresize, imshow
import numpy as np
from time import time

from PIL import Image # To resize the cover images.

# from hashlib import md5
# import glob # grab only jpegs
import warnings
warnings.filterwarnings('ignore')

from myfunk import *

%matplotlib inline

Using TensorFlow backend.


In [2]:
def remove_covers(df, files, pth):
    for f_name in files:
        try:
            os.remove(pth+f_name)
        except: 
            pass
    imfiles = [f for f in glob.glob(pth+"*.jpg")]
    df = df.loc[~df.cover.isin(files)]
    
    if df.shape[0] == len(imfiles):
        print('Success!')
    else: 
        print('Ooops something went wrong in function remove_covers', data.shape, len(image_files))
    return df, imfiles

In [3]:
# Check Path

project_name = 'book_cover_recommender'

if not os.getcwd().endswith(project_name):
    try: 
        ind = os.getcwd().index(project_name)
        idx = len(os.getcwd()) - (ind+len(project_name))
        path = os.getcwd()[:-idx]
        print(path)
        os.chdir(path)
    except:
        print('{} not found in path'.format(project_name))
else: 
    path = os.getcwd()
    print(path)

pth = "/Users/iZbra1/Documents/K2DS/Projects/book_cover_recommender/data/raw/covers/"

# Variables
image_files = [f for f in glob.glob(path+"/data/raw/covers/*.jpg")]
print(len(image_files))

data = pd.read_csv(path+"/data/raw/books.csv", index_col=0)
print(data.shape)
data.head(2)

/Users/iZbra1/Documents/K2DS/Projects/book_cover_recommender
8939
(8988, 8)


,isbn,authors,year,title,average_rating,ratings_count,image_url,cover
book_id,,,,,,,,
1,439023483,suzanne collins,2008,the hunger games the hunger games 1,4.34,4780653,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1 by suzanne...
2,439554934,jk rowling mary grandpre,1997,harry potter and the sorcerers stone harry pot...,4.44,4602479,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...


#### Removing unwanted data

In [4]:
data.loc[(data.year==0)].shape
# for inst in data.loc[(data.year==0)].title.values:
#     print(inst)

(49, 8)

In [5]:
# There are values where the year is 0, for now we'll remove them. 
delete_covers = data.loc[data.year==0].cover.values
data = data.loc[~(data.year==0)]
data, image_files = remove_covers(data, delete_covers, pth)


Success!


#### Check for dupes

In [10]:
data.shape

(8939, 8)

In [8]:
data[data.duplicated(keep=False)].shape

(0, 8)

In [7]:
data.drop_duplicates().shape

(8939, 8)

In [11]:
# data[data.duplicated(keep=False)]
data = data.drop_duplicates()
print(data.shape)

(8939, 8)


In [9]:
data.head(2)

,isbn,authors,year,title,average_rating,ratings_count,image_url,cover
book_id,,,,,,,,
1,439023483,suzanne collins,2008,the hunger games the hunger games 1,4.34,4780653,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1 by suzanne...
2,439554934,jk rowling mary grandpre,1997,harry potter and the sorcerers stone harry pot...,4.44,4602479,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...


In [12]:
# Remove average_rating and ratings_count columns and cover_exists? 
data.drop(['average_rating','ratings_count'], axis=1, inplace=True)

### Preprocess Image Files

In [13]:
### Make sure all downloaded images are encoded in RGB ~ 18s

delete_covers = []

for img_name in image_files: 
    try:
        img = imread(img_name)
    except:
        print('\nImage not found: ', img_name[77:])
        pass
    
    if len(img.shape) != 3:
        delete_covers.append(img_name[77:])
    elif img.shape[2] != 3:
        delete_covers.append(img_name[77:])

len(delete_covers)

20

In [14]:
data, image_files = remove_covers(data, delete_covers, pth)

Success!


In [15]:
data.shape, len(image_files)

((8919, 6), 8919)

In [16]:
def get_hash(image_files):
    ds_dict={}
    duplicates =[]
    hash_ds=[]
    for image in image_files:
        # Change to gray, resize to array (1,900) and compute the intensity gradient.
        ds=preprocess(image)
        # Hash_ds is a list of lists, where each list is "ds"
        hash_ds.append(ds)
        # Hash the ds variable to hexa to be used next as a dict key
        filehash = md5(ds).hexdigest()
        if filehash not in ds_dict:
            ds_dict[filehash] = image
        else:
            duplicates.append((image,ds_dict[filehash]))
    return duplicates, ds_dict, hash_ds

In [17]:
def preprocess(img):
    '''Function that gets an image filename, loads it and preprocess it:
        - Converts it to grayscale
        - Resizes it and flattens it by row and columns
        - Calculates the px to px change in intensity
        Returns a 3d array of the preprocessed image'''
    try:
        img = imread(img)
    except:
        print('\nImage not found: ', img[77:])
        pass
    # Take the average of the 3 channels to convert to grayscale
    # Take care of any image with more than 3 channels
    if img.shape[2] != 3:
        gray = np.average(img, weights = [0.299, 0.587, 0.114, 0.5], axis=2)
    else:
        gray = np.average(img, weights = [0.299, 0.587, 0.114], axis=2)

    ## ------ difference_score
    # Resize
    height=30
    width=30
    # Flatten by row
    row_res = cv2.resize(gray, (height, width), interpolation=cv2.INTER_AREA).flatten()
    # Flatten by col
    col_res = cv2.resize(gray, (height, width), interpolation=cv2.INTER_AREA).flatten("F")

    # Gradient direction based on intensity
    # Difference is a 1-D boolean array
    # where True refers to a change in intensity from 1 px to another
    difference_row = np.diff(row_res)
    difference_col = np.diff(col_res)
    # Look for change in intensities
    # True for it does, False for it doesnt change
    difference_row = difference_row > 0
    difference_col = difference_col > 0
    difference = np.vstack((difference_row, difference_col)).flatten()

    return difference

In [18]:
## --- Remove cover dupes (a.k.a covers that were not found and therefore have a blank image) ~ 30s

# Any little change in intensity yields two hashes 
duplicates, ds_dict, hash_ds = get_hash(image_files)


In [19]:
def consolidate_dupes(df, duplicates):
    '''Function that receives a path to the books files
    an a list of tuples as duplicates
    It will select unique books from the list of tuples'''

    dupes = []
    # Comb the duplicates and get the unique titles
    for file_names in duplicates:
        if file_names[0] not in dupes:
            dupes.append(file_names[0][77:])

        if file_names[1] not in dupes:
            dupes.append(file_names[1][77:])
    return dupes

In [20]:
if len(duplicates)>0:
    dupes = consolidate_dupes(data, duplicates)
    data, image_files = remove_covers(data, dupes, pth)
    print('Removed duplicates.', data.shape[0], ' books remaining ..')
else: 
    print('All covers are unique')

Success!
Removed duplicates. 5836  books remaining ..


#### Sanity Checks

In [22]:
## ---- Check if all instances in data dataframe have their book covers. 
data['cover_exists'] = data.cover.apply(lambda x: os.path.isfile(pth+x))

try: 
    assert data.loc[data.cover_exists==False].shape[0] == 0
    print('Sanity check passed!')
except AssertionError as e:
    print('Number of images doesnt correspond to the number of books in the dataset', type(e))

# Sanity check
image_files = [f for f in glob.glob(pth+'*.jpg')]
try:
    assert len(image_files) == data.shape[0]
    print('Sanity check #2 passed!')
except AssertionError: 
    print('Number of cover images in storage:', len(image_files), '\nNumber of instances:', data.shape[0])
data.drop(['cover_exists'], axis=1, inplace=True)

Sanity check passed!
Sanity check #2 passed!


In [23]:
data.to_csv(path+"/data/interim/books.csv")
data.to_csv(path+"/data/processed/books.csv")

#### Ratings 

In [24]:
# Let's drop those ratings to books that are no longer in the books.csv
rat = pd.read_csv(path+"/data/raw/ratings.csv")
print('Original datasize:',rat.shape)
rat = rat.loc[rat.book_id.isin(data.index)]
print('Updated datasize:',rat.shape)

# COLD START: Remove the user that only rated 1 books 
rpu = rat[['user_id','rating']].groupby('user_id').count()
rat = rat.loc[rat.user_id.isin(rpu.loc[rpu.rating>1].index.values)]
# Check for dupes
rat = rat.drop_duplicates()
# Get the users available
av_users = rat.user_id.unique() 
print('Final datasize:',rat.shape)

rat.to_csv(path+"/data/interim/ratings.csv")
rat.to_csv(path+"/data/processed/ratings.csv")


Original datasize: (5654857, 4)
Updated datasize: (4312842, 4)
Final datasize: (4312833, 4)


#### To_read

In [25]:
# Let's drop those ratings to books that are no longer in the books.csv
tr = pd.read_csv(path+"/data/raw/to_read.csv")
print('Original datasize:',tr.shape)
tr = tr.loc[(tr.book_id.isin(data.index))&(tr.user_id.isin(av_users))]
print('Updated datasize:',tr.shape)

tr.to_csv(path+"/data/interim/to_read.csv")
tr.to_csv(path+"/data/processed/to_read.csv")

Original datasize: (862323, 3)
Updated datasize: (686447, 3)


### Preprocess data

In [26]:
# Add average rating and total ratings
data = pd.merge(data, 
         rat[['book_id','rating']].groupby('book_id').agg({"rating": ["mean","count"]}),  
         right_index=True, left_index=True)
# Rename columns
dc = list(data.columns.values[:-2]) 
dc.extend(["avg_rating", "tot_rating"])
data.columns = dc
data.head(2)

,isbn,authors,year,title,image_url,cover,avg_rating,tot_rating
book_id,,,,,,,,
1,439023483,suzanne collins,2008,the hunger games the hunger games 1,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1 by suzanne...,4.279707,22806
2,439554934,jk rowling mary grandpre,1997,harry potter and the sorcerers stone harry pot...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...,4.351350,21850


### Preprocess the image covers

In [27]:
os.getcwd()

'/Users/iZbra1/Documents/K2DS/Projects/book_cover_recommender'

In [29]:
# Get the size of the covers
data['cover_size'] = data.cover.apply(lambda f: imread(pth+f).shape)



In [30]:
# Height vs width
data['height'] = data.cover_size.apply(lambda x: int(x[0]))
data['width'] = data.cover_size.apply(lambda x: int(x[1]))

# Drop image_url and cover_size
data.drop(['image_url','cover_size'], axis=1, inplace=True)
# Save dataframe for EDA 
data.to_csv(path + "/data/interim/books.csv")
print(data.shape)
data.head()

(5836, 9)


,isbn,authors,year,title,cover,avg_rating,tot_rating,height,width
book_id,,,,,,,,,
1,439023483,suzanne collins,2008,the hunger games the hunger games 1,the hunger games the hunger games 1 by suzanne...,4.279707,22806,146,98
2,439554934,jk rowling mary grandpre,1997,harry potter and the sorcerers stone harry pot...,harry potter and the sorcerers stone harry pot...,4.351350,21850,147,98
3,316015849,stephenie meyer,2005,twilight twilight 1,twilight twilight 1 by stephenie meyer.jpg,3.214341,16931,147,98
4,61120081,harper lee,1960,to kill a mockingbird,to kill a mockingbird by harper lee.jpg,4.329369,19088,151,98
5,743273567,f scott fitzgerald,1925,the great gatsby,the great gatsby by f scott fitzgerald.jpg,3.772224,16604,153,98


---